
## Imports

In [100]:
%reload_ext autoreload
%autoreload 2
from merkle_tree import MerkleTree
from channel import Channel
from hash import my_hash
from pprint import pprint

In [101]:
# Test

from pprint import pprint

m = MerkleTree([1, 2, 3, 4, 5, 6, 7, 8])
pprint([[b[:4] for b in a] for a in m.tree])
print(m.root)
pprint([a[:4] for a in m.get_path(0)])

[['6b86', 'd473', '4e07', '4b22', 'ef2d', 'e7f6', '7902', '2c62'],
 ['33b6', '1365', '4358', 'ada1'],
 ['85df', 'e0e2'],
 ['c274']]
c27450cd3fd4df029145f3437ae9c381e0ae55e8400de06cb973005b36d7b222
['6b86',
 ('d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35', 'right'),
 ('13656c83d841ea7de6ebf3a89e0038fea9526bd7f686f06f7a692343a8a32dca', 'right'),
 ('e0e2d0cec0ef7e8fc458e516dfde82890c183431a3f9efae9e4693fc23dfa36a', 'right')]


### Fibonacci function

In [102]:
def fibonacci(a=1, size=1001) -> list[int]:
    fib_list = [1, a]
    for _ in range(2, size):
        fib_list.append(fib_list[-1] + fib_list[-2])
    return fib_list

In [103]:
print(fibonacci())

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280099, 53316291173, 86267571272, 139583862445, 225851433717, 365435296162, 591286729879, 956722026041, 1548008755920, 2504730781961, 4052739537881, 6557470319842, 10610209857723, 17167680177565, 27777890035288, 44945570212853, 72723460248141, 117669030460994, 190392490709135, 308061521170129, 498454011879264, 806515533049393, 1304969544928657, 2111485077978050, 3416454622906707, 5527939700884757, 8944394323791464, 14472334024676221, 23416728348467685, 37889062373143906, 61305790721611591, 99194853094755497, 160500643816367088, 259695496911122585, 420196140727489673, 679891637638612258, 110008777836

### Polynomial f(x)

In [104]:
P: int =3221225473
F = GF(P)
N = 8192

g = F(1734477367)
gamma = g ** 8
gamma_group = [gamma ** i for i in range(1001)]

fibonacci_seq = fibonacci()

R.<x> = PolynomialRing(F)

f = R.lagrange_polynomial(zip(gamma_group, fibonacci_seq))

### Bit Reverse Order

In [105]:
def bit_reverse(n, width):
    rev = 0  # This will store the reversed bits
    for i in range(width):
        rev = (rev << 1) | (n & 1)  # Shift left and take LSB of n
        n >>= 1  # Shift n to the right
    return rev


def bit_reverse_permutation(N):
    """Generate bit-reversed order for N elements."""
    width = int(math.log(N, 2))
    return [bit_reverse(i, width) for i in range(N)]


bit_reverse_order = bit_reverse_permutation(N)

### w < g >

In [106]:
w = F.multiplicative_generator()

g_group = [g ** i for i in range(N)]

wg = [w * gi for gi in g_group]
wg_merkle_order = [wg[i] for i in bit_reverse_order]

### Commit f(x) on LDE

In [107]:
channel = Channel(F, N)

f_wg = [f(x) for x in wg_merkle_order]

merkle_f_wg = MerkleTree(f_wg)

channel.send({'title': 'f(x) on LDE', 'data': merkle_f_wg.root})

### Polynomial h(x)

In [108]:
d_x = 1
for gamma_i in gamma_group[:-2]:
    d_x = d_x * (x - gamma_i)

F_x = f(x) + f(gamma * x) - f((gamma**2) * x)

h_x = F_x // d_x

In [109]:
h_wg = [F_x(x) / d_x(x) for x in wg_merkle_order]

# Sanity check
h_wg_by_poly = [h_x(x) for x in wg_merkle_order]
print(h_wg_by_poly[:10])
print(h_wg[:10])
assert(h_wg == h_wg_by_poly)

[1860826706, 1845649622, 290025805, 195225050, 2156032599, 1550443729, 1756057239, 1950419089, 356807959, 128442896]
[1860826706, 1845649622, 290025805, 195225050, 2156032599, 1550443729, 1756057239, 1950419089, 356807959, 128442896]


### Boundary Constraints

In [110]:
Y = fibonacci_seq[-1]

t0_x = (f(x) - 1) / (x - gamma ** 0)
t1_x = (f(x) - Y) / (x - gamma ** 1000)

In [111]:
t0_wg = [(f(i) - 1) / (i - gamma ** 0) for i in wg_merkle_order]
t1_wg = [(f(i) - Y) / (i - gamma ** 1000) for i in wg_merkle_order]

# Sanity check
t0_x_by_poly = [t0_x(x) for x in wg_merkle_order]
t1_x_by_poly = [t1_x(x) for x in wg_merkle_order]
print(t0_x_by_poly[:10])
print(t0_wg[:10])
print("")
print(t1_x_by_poly[:10])
print(t1_wg[:10])
assert(t0_wg == t0_x_by_poly)
assert(t1_wg == t1_x_by_poly)

[2384530853, 599226013, 2071111314, 1402510381, 2026908630, 951059204, 1906847992, 1092764146, 3083267015, 51241944]
[2384530853, 599226013, 2071111314, 1402510381, 2026908630, 951059204, 1906847992, 1092764146, 3083267015, 51241944]

[2395950328, 2696799910, 124207998, 3010185695, 1771367485, 510712279, 521057469, 2771188778, 1200373265, 956436872]
[2395950328, 2696799910, 124207998, 3010185695, 1771367485, 510712279, 521057469, 2771188778, 1200373265, 956436872]


### Composition Polynomial

In [112]:
beta_0 = channel.receive_random_field_element()
beta_1 = channel.receive_random_field_element()
beta_2 = channel.receive_random_field_element()

cp0_x = beta_0 * h_x + beta_1 * t0_x + beta_2 * t1_x

In [113]:
cp0_wg = [beta_0 * h_wg[i] + beta_1 * t0_wg[i] + beta_2 * t1_wg[i]
          for i in range(len(wg))]

# Sanity check
cp0_x_by_poly = [cp0_x(x) for x in wg_merkle_order]
print(cp0_x_by_poly[:10])
print(cp0_wg[:10])
assert(cp0_wg == cp0_x_by_poly)

[444028236, 771215598, 3165881204, 396364446, 689583236, 153884535, 2130846259, 1461295508, 2813887909, 3170606964]
[444028236, 771215598, 3165881204, 396364446, 689583236, 153884535, 2130846259, 1461295508, 2813887909, 3170606964]


### FRI

In [123]:
fri_layers = []
degree = R(cp0_x).degree()
# if degree % 2 != 0:
#     degree += 1

curr_cp = [cp0_x(x) for x in wg_merkle_order]
merkle_curr_cp = MerkleTree(curr_cp)
channel.send({
    "title": f"commit for cp 0",
    "data": merkle_curr_cp.root,
})
z = channel.receive_random_field_element()
y_0 = f(z)
channel.send({
    "title": "y_0",
    "data": y_0,
})
y_1 = f(gamma * z)
channel.send({
    "title": "y_1",
    "data": y_1,
})
y_2 = f((gamma**2) * z)
channel.send({
    "title": "y_2",
    "data": y_2,
})
y_3 = cp0_x(z)
channel.send({
    "title": "y_3",
    "data": y_3,
})
alpha_0 = channel.receive_random_field_element()
alpha_1 = channel.receive_random_field_element()
alpha_2 = channel.receive_random_field_element()
alpha_3 = channel.receive_random_field_element()
fri_first_layer_poly = alpha_0 * (f - y_0) // (x - z) + alpha_1 * (f - y_1) // (x - gamma * z) + alpha_2 * (f - y_2) // (x - (gamma**2) * z) + alpha_3 * (cp0_x - y_3) // (x - z)
fri_first_layer_on_wg_by_poly = [fri_first_layer_poly(x) for x in wg_merkle_order]
fri_first_layer_on_wg = [
    alpha_0 * (f(x) - y_0) / (x - z) + alpha_1 * (f(x) - y_1) / (x - gamma * z) + alpha_2 * (f(x) - y_2) / (x - (gamma**2) * z) + alpha_3 * (cp0_x(x) - y_3) / (x - z) for x in wg_merkle_order 
]
assert(fri_first_layer_on_wg == fri_first_layer_on_wg_by_poly)
while degree > 0:
    # Commit curr layer
    merkle_curr_cp = MerkleTree(curr_cp)
    fri_layers.append((curr_cp, merkle_curr_cp))
    channel.send({
        "title": f"commit for layer {len(fri_layers) - 1} of FRI",
        "data": merkle_curr_cp.root,
    })
    query_x = channel.receive_random_field_element()
    curr_cp = [((curr_cp[i] + curr_cp[i + 1]) / 2) +
               query_x * (curr_cp[i] - curr_cp[i + 1]) / (2 * wg_merkle_order[i]) for i in range(len(curr_cp))[::2]]
    degree = degree // 2

constant = curr_cp[0]
channel.send({"title:": "constant of last FRI layer",
              "data": constant,
              })

assert(all([constant == x for x in curr_cp]))

In [115]:
print(fri_layers)
print("Merkle Root")
print(curr_cp[0])

[([444028236, 771215598, 3165881204, 396364446, 689583236, 153884535, 2130846259, 1461295508, 2813887909, 3170606964, 424079901, 2173323613, 1156458944, 1849715001, 3199220210, 2706714542, 1068914929, 2664583475, 552647337, 2154136777, 1888797072, 2727305669, 503426198, 165822617, 1523395854, 2519808770, 1954356136, 432978773, 86465143, 3083563504, 811271108, 1440095167, 3068626395, 1794080746, 2821896332, 3098956694, 549192945, 2073741438, 2418562136, 297340365, 2849924807, 1764922745, 770659287, 1442301823, 3063975984, 2287185538, 59625038, 2824498140, 3026890994, 560673081, 132420943, 739925263, 947309618, 651030414, 2462977522, 2620959540, 213709583, 2472934965, 1854176586, 165954883, 1946145006, 327068722, 352231727, 1227129215, 2026713743, 1275180937, 1715117643, 1411887921, 208960766, 3122574133, 1562022529, 2866925767, 1857151465, 572139098, 585320628, 1407982077, 98378259, 2133266461, 1298242721, 2580743514, 2360565670, 3203026368, 141237094, 205078958, 459308613, 870850935, 5

In [116]:
# Sanity check
print(curr_cp)

[2889524960, 2889524960, 2889524960, 2889524960, 2889524960, 2889524960, 2889524960, 2889524960]


In [117]:
pprint(channel.get_all_messages())

[{'data': 'd524fba68e1263f084afe501c715dca773c616d5a8c52303322985d32c94ea56',
  'title': 'f(x) on LDE'},
 {'data': 2415762890, 'title': 'Random Field Element'},
 {'data': 2231440058, 'title': 'Random Field Element'},
 {'data': 3022999430, 'title': 'Random Field Element'},
 {'data': '43e25e07478da78c58fc606e861c6e25fc49d83f79a138f3d6e3cc9330c09aec',
  'title': 'commit for layer 0 of FRI'},
 {'data': 279845176, 'title': 'Random Field Element'},
 {'data': 'f18af140a8ea1331305c1d16c0607a4ecd5f9233ff0ba7403ca0ae153ad9d81c',
  'title': 'commit for layer 1 of FRI'},
 {'data': 2383407487, 'title': 'Random Field Element'},
 {'data': 'ce567c49ea18e7c99cd3523761e0148f74d52cedf28fb2c2670f55cdd0475af9',
  'title': 'commit for layer 2 of FRI'},
 {'data': 2109765792, 'title': 'Random Field Element'},
 {'data': 'f55115783536aafde85471ed468e6646753dfdc2f204854da9499d4a1deeaaf6',
  'title': 'commit for layer 3 of FRI'},
 {'data': 92411445, 'title': 'Random Field Element'},
 {'data': '801febf74e368a8375a

### Decommit Phase

# TIOTA


## Decommit Phase

In [118]:
def send_f_of_x_and_gammas(idx_of_x, idx_of_gamma_x, idx_of_gamma_2_x):
    channel.send({
        'title': 'f of x',
        'data': f_wg[idx_of_x],
    })
    channel.send({
        'title': 'path of f of x',
        'data': merkle_f_wg.get_path(idx_of_x),
    })
    channel.send({
        'title': 'f of gamma x',
        'data': f_wg[idx_of_gamma_x],
    })
    channel.send({
        'title': 'path of f of gamma x',
        'data': merkle_f_wg.get_path(idx_of_gamma_x),
    })
    channel.send({
        'title': 'f of gamma squared x',
        'data': f_wg[idx_of_gamma_2_x],
    })
    channel.send({
        'title': 'path of f of gamma squared x',
        'data': merkle_f_wg.get_path(idx_of_gamma_2_x),
    })

def send_cp_layer(idx_of_x, idx_of_minus_x, cp, cp_merkle, i):
    assert(cp_merkle.get_path(idx_of_x)[2:] == cp_merkle.get_path(idx_of_minus_x)[2:])
    channel.send({
        'title': f'path of cp of x - layer {i}',
        'data': cp_merkle.get_path(idx_of_x),
    })
    channel.send({
        'title': f'cp of -x - layer {i}',
        'data': cp[idx_of_minus_x],
    })
    channel.send({
        'title': f'path of cp of -x - layer {i}',
        'data': cp_merkle.get_path(idx_of_minus_x),
    })

def query(idx):
    idx_of_x = bit_reverse_order.index(idx)
    idx_of_gamma_x = bit_reverse_order.index(idx + 8)
    idx_of_gamma_2_x = bit_reverse_order.index(idx + 16)
    send_f_of_x_and_gammas(idx_of_x, idx_of_gamma_x, idx_of_gamma_2_x)
    for i, (cp, cp_merkle) in enumerate(fri_layers):
        idx_of_minus_x = idx_of_x ^^ 1
        send_cp_layer(idx_of_x, idx_of_minus_x, cp, cp_merkle, i)
        idx_of_x //= 2

In [119]:
channel.start_decommit()
for i in range(8):
    random_idx = channel.receive_random_query()
    query(random_idx)

In [120]:
def check_merkle_path(data, path, root):
    curr = my_hash(data)
    if curr != path[0]:
        return False
    for sibling, direction in path[1:]:
        if direction == 'left':
            curr = my_hash(sibling + curr)
        else:
            curr = my_hash(curr + sibling)
    return curr == root

# Sanity
assert(check_merkle_path(f_wg[4000], merkle_f_wg.get_path(4000), merkle_f_wg.root))

# Verifier

In [121]:
channel.receive_decommitment()

[{'title': 'Random Query', 'data': 1256},
 {'title': 'f of x', 'data': 229727548},
 {'title': 'path of f of x',
  'data': ['3458e523883bd2b42a35386d3b336b7445a88401773fc3503a0f5d457841e5a4',
   ('a794cc76cbeba388c67a5a5001ae1114a86b1fa9e3a350050831052efe975d37',
    'right'),
   ('3f2edc1fd7999d97c552f599b3417a54691882e69cf50e37ab22073718857136',
    'right'),
   ('16e53446fc345088a4f9ef55f718253535c602390a837666aa055e98c26faba8',
    'left'),
   ('9a497f58cdeb048eee57af8f86a83b64cbe3e5f32058bbe624bcbef2bae9b717',
    'right'),
   ('471cf674f11e8c2544a25e6e78b6b177c2863cb6bf44bbebebf15054894d0284',
    'right'),
   ('fd593c245631526a6149347ffe073d71770c581a4c8ded5528a416f1bc115502',
    'left'),
   ('08abe0e963e69f537e650e02beea628fa1a975917fa111e35e3fd48e2bbf229c',
    'left'),
   ('ecf3dc3873523de665260c667272a0c8c534c39f320900e19cd232831b12ecd3',
    'left'),
   ('cee51750d1c52e820b73a80b65f0cbf78d1ed57bfc2d35f427d8c06eab91f492',
    'right'),
   ('30a6c0404e508b31b8bc8729c71cc9e03e

In [ ]:
P: int = 3221225473
F = GF(P)
N = 8192
g = F(1734477367)
gamma = g ** 8
Y = fibonacci_seq[-1]

# Initial tree
commitment = channel.receive_commitment()
decommitment = channel.receive_decommitment()
random_idx = decommitment[0]['data']
f_x = decommitment[1]['data']
path_of_f_x = decommitment[2]['data']
f_gamma_x = decommitment[3]['data']
path_of_f_gamma_x = decommitment[4]['data']
f_gamma_2_x = decommitment[5]['data']
path_of_f_gamma_2_x = decommitment[6]['data']
root = commitment[0]['data']
assert(check_merkle_path(f_x, path_of_f_x, root))
assert(check_merkle_path(f_gamma_x, path_of_f_gamma_x, root))
assert(check_merkle_path(f_gamma_2_x, path_of_f_gamma_2_x, root))

# CP 0
beta_0 = commitment[1]['data']
beta_1 = commitment[2]['data']
beta_2 = commitment[3]['data']

F_x = f_x + f_gamma_x - f_gamma_2_x
g_pow = g ** random_idx
query_x = w * g_pow
d_x_denominator = 1
for gamma_i in [gamma ** i for i in range(999, 1024)]:
    d_x_denominator = d_x_denominator * (query_x - gamma_i)
d_x = (query_x ** 1024 - 1) / d_x_denominator

h_x = F_x / d_x
t0_x = (f_x - 1) / (query_x - gamma ** 0)
t1_x = (f_x - Y) / (query_x - gamma ** 1000)
cp_x = beta_0 * h_x + beta_1 * t0_x + beta_2 * t1_x

commitment_idx = 4
decommitment_idx = 7
while commitment_idx < len(commitment) - 1:
    path_of_cp_x = decommitment[decommitment_idx]['data']
    cp_minus_x = decommitment[decommitment_idx + 1]['data']
    path_of_cp_minus_x = decommitment[decommitment_idx + 2]['data']

    cp_root = commitment[commitment_idx]['data']
    assert(check_merkle_path(cp_x, path_of_cp_x, cp_root)), (cp_x, path_of_cp_x, cp_root)
    assert(check_merkle_path(cp_minus_x, path_of_cp_minus_x, cp_root))

    random_x = commitment[commitment_idx + 1]['data']
    query_x = w * g_pow
    cp_x = (cp_x + cp_minus_x) / 2 + random_x * (cp_x - cp_minus_x) / (2 * query_x)
    
    g_pow = g_pow ** 2

    commitment_idx += 2
    decommitment_idx += 3

assert(cp_x == commitment[commitment_idx]['data'])